In [ ]:
#download pySpark

!java -version

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

!pip install pyspark

!pip install lifetimes

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_362"
OpenJDK Runtime Environment (build 1.8.0_362-8u362-ga-0ubuntu1~20.04.1-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 25.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=83f620c3085803fefbfcfdf059f731b3eed8eb03fe85dec97d3ffd223b5acc94
  Stored in directory: /root/.cache/pip/wheels/6c/e3

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime, time
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf

from pyspark.sql.functions import unix_timestamp, col, to_date, struct, abs,when, round, desc, concat, countDistinct

from pyspark.sql import Row
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import max as max_

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("App_Name") \
    .config("spark.driver.memory", "64g") \
    .getOrCreate()

In [ ]:
#data load

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/NDSU_capstone/2.Raw_data/2.Transaction"

In [ ]:
### Read all files in Transaction folder
df = spark.read.csv(path,header=True)

In [ ]:
# df.count()

In [ ]:
# df.show(3)

In [ ]:
df2 = df.drop('RTXNNBR','PARENT_ACCT_KEY','RTXN_TYPE','PARENT_TRAN_NBR','MINOR_TYP_CD')
# df2.show(4)

In [ ]:
df2_abs = df2.withColumn('abs_AMT',abs(df2.RTXN_AMT))
# df2_abs.show()

In [ ]:
# Groupby with RTXN_TYPE_CD  with sum()

df2_raw = df2_abs.groupBy('RTXN_TYPE_CD').agg(_sum('abs_AMT').alias("Total Fee")).sort(desc("Total Fee"))

In [ ]:
# df2_raw.coalesce(1).write.csv('/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/Transaction/transaction_type_byAMT')

In [ ]:
# # PySpark groupBy() count
# df2_count = df2.groupBy("RTXN_TYPE_CD").count().sort(col("count").desc())
# df2_count.show()
# df2_count.coalesce(1).write.csv('/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/Transaction/transaction_type_byCOUNT')

In [ ]:
df_date = df2.filter(df2['ACT_DATE_TIME'] >= F.lit('2019-01-01 00:00:00'))
df_date.show(vertical= True)

In [ ]:
# df_date.count()

In [ ]:
df3 = df_date.filter(df_date.RTXN_CNT  == 1)
# df3.count()

In [ ]:
df4 = df3.filter(df3.MAJOR_TYPE  == "CK")
# df4.count()

In [ ]:
df5 = df4.filter(df4.RTXN_TYPE_CD.isin(['PWTH','XDEP']))
# df5.count()

In [ ]:
df5.show()

+--------------------+--------+------------------+--------+------------+----------+-------------------+-------------------+--------------------+--------------------+--------------------+-----------------+--------+
|             ACCTKEY|RTXN_AMT|BAL_AFTER_RTXN_AMT|RTXN_CNT|RTXN_TYPE_CD|MAJOR_TYPE|      ACT_DATE_TIME|           EFF_DATE|SIGNED_BASED_TXN_FLG|              BRANCH|                 TXT|RECURRING_TRAN_YN|SICSUBCD|
+--------------------+--------+------------------+--------+------------+----------+-------------------+-------------------+--------------------+--------------------+--------------------+-----------------+--------+
|2c99b686a3f99afc4...|   864.0|            903.64|       1|        XDEP|        CK|2020-11-24 03:53:00|2020-11-24 00:00:00|                null|766d856ef1a6b02f9...|Job Service ND  -...|                N|    null|
|fabcec8287b2a4af7...|   545.0|          199064.2|       1|        XDEP|        CK|2020-11-16 04:09:45|2020-11-16 00:00:00|                null|

In [ ]:
# df5.agg(F.min("ACT_DATE_TIME"), F.max("ACT_DATE_TIME")).show()

In [ ]:
#import account file

In [ ]:
path_acct = '/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/acc_123122.csv'

In [ ]:
data = spark.read.csv(path_acct,header=True)

In [ ]:
type(data)

pyspark.sql.dataframe.DataFrame

In [ ]:
# data.show()

In [ ]:
# acc_df = data[['account_key','customer_type','Household_key']].copy()
# acc_df1 = acc_df.drop_duplicates(subset=['account_key'], keep='first')
# print(len(acc_df1))

In [ ]:
# acc_sdf = spark.createDataFrame(acc_df1)

In [ ]:
# type(acc_sdf)

In [ ]:
# acc_sdf.show()

In [ ]:
flat_df = df5.join(data,df5.ACCTKEY == data.account_key,"left")

In [ ]:
flat_df.show(4)

In [ ]:
flat_df_final = flat_df.filter(flat_df.customer_type  == "Personal")

In [ ]:
flat_df_final = flat_df_final.drop('account_key')

In [ ]:
# flat_df_final.count()

In [ ]:
flat_df_final.show(4)

In [ ]:
flat_df_final.write.parquet("/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/filter_transaction_110119.parquet")

In [ ]:
!wget -P /content/drive/MyDrive/NDSU_capstone/3.proceeded_data http://134.129.122.10:8888/files/Trang/all_transaction.zip?download=1

--2023-03-30 19:09:32--  http://134.129.122.10:8888/files/Trang/all_transaction.zip?download=1
Connecting to 134.129.122.10:8888... connected.
HTTP request sent, awaiting response... 302 Found
Location: /login?next=%2Ffiles%2FTrang%2Fall_transaction.zip%3Fdownload%3D1 [following]
--2023-03-30 19:09:32--  http://134.129.122.10:8888/login?next=%2Ffiles%2FTrang%2Fall_transaction.zip%3Fdownload%3D1
Reusing existing connection to 134.129.122.10:8888.
HTTP request sent, awaiting response... 200 OK
Length: 9199 (9.0K) [text/html]
Saving to: ‘/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/all_transaction.zip?download=1’

all_transaction.zip 100%[===================>]   8.98K  --.-KB/s    in 0s      

2023-03-30 19:09:32 (119 MB/s) - ‘/content/drive/MyDrive/NDSU_capstone/3.proceeded_data/all_transaction.zip?download=1’ saved [9199/9199]

